In [1]:
%%capture
!pip install papermill

In [2]:
import ipywidgets as widgets
import papermill as pm

from IPython.display import display

In [3]:
dict_names = {1: 'Especial', 2: 'Pradera UPB FS', 3: 'PANZENU', 5: 'DORADO', 6: 'KM30',
        7: 'SANTANDER', 8: 'TAMBO CIRCUNVALAR', 9: 'MOGAMBO 22', 12: '6 DE MARZO',
        13: 'CARRIZAL', 17: 'PRADERA 27', 18: 'Furatena', 20: 'KM 15', 21: 'SANTA LUCIA',
        24: 'TERMINAL - AEROPUERTO', 26: 'RANCHO GRANDE', 27: 'SABANAL', 34: 'LETICIA',
        36: 'AGUAS VIVAS', 37: 'CAÑO VIEJO', 97: 'Auditorias RE'}


In [4]:
lineas = widgets.SelectMultiple(
    options=dict_names.values(),
    description='Rutas:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
#     icons=['check'] * 3
)
nodos_ruta = widgets.BoundedIntText(
    value=12,
    min=0,
    max=30,
    step=1,
    description='# Paradas',
    disabled=False
)
buses_disponibles = widgets.IntSlider(
    value=32,
    min=1,
    max=50,
    step=1,
    description='# Buses:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
capacidad_bus = widgets.IntSlider(
    value=50,
    min=10,
    max=50,
    step=1,
    description='# Pasajeros:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
agrupacion_minutos = widgets.IntSlider(
    value=15,
    min=5,
    max=60,
    step=5,
    description='minutos (t):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)


display(lineas)
display(nodos_ruta)
display(buses_disponibles)
display(capacidad_bus)
display(agrupacion_minutos)

SelectMultiple(description='Rutas:', options=('Especial', 'Pradera UPB FS', 'PANZENU', 'DORADO', 'KM30', 'SANT…

BoundedIntText(value=12, description='# Paradas', max=30)

IntSlider(value=32, continuous_update=False, description='# Buses:', max=50, min=1)

IntSlider(value=50, continuous_update=False, description='# Pasajeros:', max=50, min=10)

IntSlider(value=15, continuous_update=False, description='minutos (t):', max=60, min=5, step=5)

In [5]:
button_previsualize = widgets.Button(description='Visualizar Escenario')
button_excecute = widgets.Button(description='Ejecutar Escenario')
output = widgets.Output()
output_excecute = widgets.Output()

display(button_previsualize, output)
display(button_excecute, output_excecute)

def previsualice_run(b):
    with output:
        print('======= Nuevo click Visualizar Escenario =======')
        print(f'Lineas: {lineas.value}')
        print(f'Num Nodos/linea: {nodos_ruta.value}')
        print(f'Buses Disponibles : {buses_disponibles.value}')
        print(f'Capacidad Bus: {capacidad_bus.value}')
        print(f'Minutos (t): {agrupacion_minutos.value}')
        print(f'Maximos periodos posibles (t_max): {int(960/agrupacion_minutos.value)}')

def schedule_line(b):
    with output:
        print('>>> Ejecutando agendamiento con los siguientes parametros:')
        print(f'Lineas: {lineas.value}')
        print(f'Num Nodos/linea: {nodos_ruta.value}')
        print(f'Buses Disponibles : {buses_disponibles.value}')
        print(f'Capacidad Bus: {capacidad_bus.value}')
        print(f'Minutos (t): {agrupacion_minutos.value}')
        print(f'Maximos periodos posibles (t_max): {int(960/agrupacion_minutos.value)}')
        print('Ejecutando por favor espere . . .')
        for line_name in lineas.value:
            print(f'--> {line_name}')
            print('1. Inicio - pronostico de demanda ...')
            pm.execute_notebook(
               './data/template.ipynb',
               f'./outputs/notebooks/demand/{line_name}.ipynb',
               parameters=dict(
                   ruta=line_name,
                   agrupacion_minutos=agrupacion_minutos.value,
                   buses_disponibles=buses_disponibles.value,
                   capacidad_bus=capacidad_bus.value,
                   t_max=int(960/agrupacion_minutos.value)
               )
            )
            print('1. Fin - pronostico de demanda')
            print('2. Inicio - ejecución modelo ...')
            pm.execute_notebook(
               './model/ani.ipynb',
               f'./outputs/notebooks/model/{line_name}.ipynb',
               parameters=dict(
                   ruta=line_name,
                   agrupacion_minutos=agrupacion_minutos.value,
                   buses_disponibles=buses_disponibles.value,
                   capacidad_bus=capacidad_bus.value,
                   t_max=int(960/agrupacion_minutos.value)
               )
            )
            print('2. Fin ejecución modelo - ejecución modelo ...')
        print('Todo se ejecuto exitosamente!')
        print('Para ver los notebooks ejecutados dirigase a ./outputs/model_notebooks')
        
        

button_previsualize.on_click(previsualice_run)
button_excecute.on_click(schedule_line)

Button(description='Visualizar Escenario', style=ButtonStyle())

Output()

Button(description='Ejecutar Escenario', style=ButtonStyle())

Output()